# 1. 데이터 읽어오기

In [ ]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense


txt_file_path = '/content/data/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담음
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said", 'Or what you threw at me Please tell me', 'Please tell me why', 'My car is in the front yard', 'And I am sleeping with my cloths on', 'I came in throught the window... Last night', 'And your... Gone', "Gone It's no suprise to me I am my own worst enemy", 'Cuz every now and then I kick the living shit out of me', 'The smoke alarm is going offf and there a cigarette', 'Still buring Please tell me why', 'My car is in the front yard', "And I'm sleeping with my clothes on", 'I came in throught the windo last night', 'And your gone', 'Gone Please tell me why', 'My car is in the front yard', 'And I am sleeping with my clothes on']


# 2. 데이터 정제

In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜀
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜀

    if idx >15: break   #  토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
        
    print(sentence)

Can we forget about the things I said when I was drunk...
I didn't mean to call you that
I can't remember what was said
Or what you threw at me Please tell me
Please tell me why
My car is in the front yard
And I am sleeping with my cloths on
I came in throught the window... Last night
And your... Gone
Gone It's no suprise to me I am my own worst enemy
Cuz every now and then I kick the living shit out of me
The smoke alarm is going offf and there a cigarette
Still buring Please tell me why
My car is in the front yard
And I'm sleeping with my clothes on
I came in throught the windo last night


In [ ]:
# 정규표현식을 이용한 필터링  
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()   
    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
     # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r'[" "]+', " ", sentence) 
     # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 

    sentence = sentence.strip()
    # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여줌
    sentence = '<start> ' + sentence + ' <end>'      
    
    return sentence

In [ ]:
# 정제된 문장을 모음
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> can we forget about the things i said when i was drunk . . . <end>',
 '<start> i didn t mean to call you that <end>',
 '<start> i can t remember what was said <end>',
 '<start> or what you threw at me please tell me <end>',
 '<start> please tell me why <end>',
 '<start> my car is in the front yard <end>',
 '<start> and i am sleeping with my cloths on <end>',
 '<start> i came in throught the window . . . last night <end>',
 '<start> and your . . . gone <end>',
 '<start> gone it s no suprise to me i am my own worst enemy <end>']

In [ ]:
# 토큰화

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 구축한 corpus로부터 Tokenizer가 사전을 자동구축함

    # tokenizer를 활용하여 모델에 입력할 데이터셋을 구축
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환
	
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공
    # corpus의 가장 긴 문장을 기준으로 시퀀스 길이를 맞춤
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, 
                                                           padding='post',
                                                           maxlen=maxlen)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  32  23 ...   0   0   0]
 [  2   5 343 ...   0   0   0]
 [  2   5  32 ...   0   0   0]
 ...
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fb2c6788790>


In [ ]:
print(tensor[:3, :10])  # 단어 사전이 어떻게 구축되었는지 확인

[[  2  32  23 447 113   6 179   5 107  46]
 [  2   5 343  15 243  10 154   7  17   3]
 [  2   5  32  15 311  40  57 107   3   0]]


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [ ]:
#마지막 토큰을 자름
src_input = tensor[:, :-1]

#앞에 start부분을 자름 
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  32  23 447 113   6 179   5 107  46   5  57 720  20  20  20   3   0
   0]
[ 32  23 447 113   6 179   5 107  46   5  57 720  20  20  20   3   0   0
   0]


# 3. 평가 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

In [ ]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (140599, 19)
Target Train:  (140599, 19)


# 4. 인공지능 만들기

In [ ]:
# 텍스트 생성 모델
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 19
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 모델 학습
history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
550/550 [==============================] - 352s 610ms/step - loss: 2.9215 - val_loss: 2.5980
Epoch 2/10
550/550 [==============================] - 339s 617ms/step - loss: 2.4771 - val_loss: 2.4032
Epoch 3/10
550/550 [==============================] - 340s 618ms/step - loss: 2.3082 - val_loss: 2.2821
Epoch 4/10
550/550 [==============================] - 340s 618ms/step - loss: 2.1762 - val_loss: 2.1926
Epoch 5/10
550/550 [==============================] - 339s 617ms/step - loss: 2.0667 - val_loss: 2.1282
Epoch 6/10
550/550 [==============================] - 340s 618ms/step - loss: 1.9342 - val_loss: 2.0706
Epoch 7/10
550/550 [==============================] - 340s 618ms/step - loss: 1.8094 - val_loss: 2.0165
Epoch 8/10
550/550 [==============================] - 340s 618ms/step - loss: 1.6832 - val_loss: 1.9717
Epoch 9/10
550/550 [==============================] - 340s 619ms/step - loss: 1.5575 - val_loss: 1.9380
Epoch 10/10
550/550 [==============================] - 340s 618m

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  133019    
                                                                 
 lstm (LSTM)                 multiple                  16941056  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  14345049  
                                                                 
Total params: 64,981,748
Trainable params: 64,981,748
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야됨
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됨 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 함
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s a monster <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he has", max_len=20)

'<start> he has a sucker <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round oh oh oh oh oh oh oh <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> she is", max_len=20)

'<start> she is a sucker <end> '

# 회고

순환신경망(RNN) 작동 방법을 제대로 이해하지 못해 이번 프로젝트를 완성하는 데 어려움이 있었다.  순환신경망을 정리하자면 모델 내에서 생성한 단어를 다시 입력으로 사용하는 순환적 특성이 있는 인공지능이다.  이런 특성을 이용해 자연어처리를 하는데 순서는 아래와 같다.  

- 문장->필터링->토큰화(단어로 쪼갬)->벡터화(단어사전)->데이터셋 분리->모델 학습->평가    

그리고 데이터가 커서 학습시킬 때 시간이 오래 걸렸었다. 그래서 epoch을 5로 하다가 10으로 늘려서 학습했더니, val_loss 값을 2.2 이하로 줄일 수 있는 모델을 만들 수 있었다. 완성된 문장도 여러 개 출력해보니 모델이 잘 학습된 것 같다.

이번 프로젝트를 진행하면 RNN의 활용사례에 관심이 생겼다.   자연어 처리 뿐만 아니라 주식 가격이나 시계열 데이터 예측 등에도 활용될 수 있을 것 같다. 